In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
# from matplotlib.ticker import FormatStrFormatter
import seaborn
import copy
import warnings
warnings.filterwarnings('ignore')

In [2]:
ntwk=pd.read_csv(
    'data/networks.csv'
)

ntwk = ntwk.drop_duplicates(subset=['network_id', 'participant_id','parent_participant_id'], keep='last') #remove duplicates based on the first three columns
ntwk=ntwk.dropna() #remove nan
# ntwk.head()

ptcp=pd.read_csv(
    'data/participants.csv'
)
# ptcp=ptcp.dropna() #remove nan
# ptcp.head()

ord=pd.read_csv(
    'data/orderings.csv'
)
trls=pd.read_csv(
    'data/trials.csv'
)
cprs=pd.read_csv(
    'data/comparisons.csv'
)

In [3]:
#ptcp_gnometrans is the copy of database participant but used for fidelity of transmission of gnome algorithm
ptcp_gnometrans=ptcp.copy()

ptcp_gnometrans.astype({'algorithm':int})
ptcp_gnometrans=ptcp_gnometrans.loc[ptcp_gnometrans['algorithm'] == 1]

ptcp_gnometrans.insert(1, "n_kids", 0) #Add a column "number of kids" and set default value to be zero
ptcp_gnometrans.insert(2, "n_algokids", 0) #Add a column "number of algorithm kids" and set default value to be zero
ptcp_gnometrans.insert(3, "p_trans", 0) #Add a column "probability of transmission fidelity" and set default value to be zero
ptcp_gnometrans.insert(4, "n_nece_compare", 0) #Add a column "number of necessary comparisons" and set default value to be zero
ptcp_gnometrans.insert(5, "s_demo", 0) #Add a column "score of demonstration" and set default value to be zero
print("number of participants who used gnome: ",ptcp_gnometrans.shape[0])
# ptcp_gnometrans.head()

number of participants who used gnome:  727


In [4]:
#Generate a dictionary {parent id:number of kids} from the database network
n_kids={} #number of "kids" each parent has
for ind in ntwk.index:
    if str(int(ntwk['parent_participant_id'][ind])) not in n_kids.keys():
        n_kids[str(int(ntwk['parent_participant_id'][ind]))]=1
    else:
        n_kids[str(int(ntwk['parent_participant_id'][ind]))]+=1

print(n_kids)
print("number of participants that have kids: ",len(n_kids.keys()))

# n_kids=dict(sorted(n_kids.items(), key=lambda x:x[1], reverse=True)) #{parent id:number of kids}sorted by number of kids

{'64': 1, '70': 4, '74': 3, '66': 1, '27': 2, '18': 2, '21': 2, '17': 6, '29': 3, '68': 3, '71': 5, '56': 6, '49': 4, '45': 1, '79': 6, '83': 3, '77': 4, '39': 7, '36': 2, '43': 2, '58': 5, '46': 1, '48': 4, '47': 3, '54': 2, '84': 1, '85': 3, '33': 3, '69': 3, '61': 2, '44': 2, '78': 2, '13': 2, '8': 3, '82': 2, '89': 4, '80': 2, '2': 5, '9': 2, '3': 4, '16': 3, '25': 3, '19': 2, '23': 2, '4': 2, '12': 3, '62': 2, '73': 2, '76': 4, '10': 3, '34': 2, '42': 2, '26': 2, '51': 5, '20': 2, '52': 2, '87': 2, '37': 3, '32': 6, '6': 1, '7': 2, '55': 2, '86': 2, '35': 2, '40': 3, '72': 2, '63': 3, '15': 1, '30': 1, '24': 2, '50': 1, '60': 1, '57': 1, '67': 1, '14': 4, '5': 2, '59': 1, '1': 2, '149': 2, '115': 2, '123': 5, '124': 3, '110': 1, '120': 6, '155': 2, '138': 2, '136': 3, '171': 1, '126': 2, '166': 2, '101': 3, '127': 2, '116': 5, '114': 2, '164': 4, '108': 3, '147': 3, '92': 3, '163': 2, '134': 6, '173': 4, '179': 3, '176': 3, '148': 2, '122': 3, '141': 3, '169': 3, '167': 1, '132': 

In [5]:
#Add number of kids  in the added column n_kids in the copy of database participants 
for ind in ptcp_gnometrans.index:
    p_id=ptcp_gnometrans['participant_id'][ind]
    if str(int(p_id)) in n_kids.keys():
        ptcp_gnometrans.at[ind,'n_kids']=n_kids[str(int(p_id))]

ptcp_gnometrans=ptcp_gnometrans.sort_values('n_kids',ascending=False)
ptcp_gnometrans.to_csv('ptcp_gnometrans.csv',index=False)
#verify
print("")
print("TO VERIFY: number of kids of participant 1861 from the dictionary n_kids:",n_kids['1861'],", which is the same as shown in the dataframe")

# ptcp_gnometrans.head(10)



TO VERIFY: number of kids of participant 1861 from the dictionary n_kids: 1 , which is the same as shown in the dataframe


In [6]:
#Generate a dictionary {parent id:number of algo_kids} from the database network and participant

p_id_gnome=ptcp_gnometrans['participant_id'].to_numpy() #list of participant id that uses gnome algorithm

n_algokids={} #number of algorithm children each parent has
for ind in ntwk.index:
    if ntwk['participant_id'][ind] in p_id_gnome:
        if str(int(ntwk['parent_participant_id'][ind])) not in n_algokids.keys():
            n_algokids[str(int(ntwk['parent_participant_id'][ind]))]=1
        else:
            n_algokids[str(int(ntwk['parent_participant_id'][ind]))]+=1


n_algokids=dict(sorted(n_algokids.items(), key=lambda x:x[1], reverse=True)) #{parent id:number of kids}sorted by number of kids
print(n_algokids)
print("number of participants using gnome that have algorithm kids: ",len(n_algokids.keys()))


{'2925': 9, '1828': 8, '1911': 8, '2004': 7, '2905': 7, '2920': 7, '3144': 7, '2546': 7, '2288': 6, '2374': 6, '2554': 6, '2757': 6, '2774': 6, '2767': 6, '2849': 6, '2902': 6, '2950': 6, '2969': 6, '3051': 6, '3075': 6, '2602': 6, '2605': 6, '2894': 6, '2888': 6, '3123': 6, '3232': 6, '3318': 6, '3173': 6, '2998': 6, '1919': 5, '2349': 5, '2493': 5, '2567': 5, '2759': 5, '2802': 5, '2825': 5, '2842': 5, '2866': 5, '2895': 5, '2923': 5, '2951': 5, '3016': 5, '3037': 5, '3074': 5, '3066': 5, '2607': 5, '3108': 5, '3134': 5, '3278': 5, '3286': 5, '3324': 5, '3153': 5, '3504': 5, '3095': 5, '3094': 5, '3092': 5, '3562': 5, '1273': 4, '1283': 4, '1850': 4, '1839': 4, '1944': 4, '1940': 4, '2078': 4, '2164': 4, '2148': 4, '2323': 4, '2428': 4, '2427': 4, '2422': 4, '2512': 4, '2064': 4, '2594': 4, '2022': 4, '2717': 4, '2741': 4, '2733': 4, '2858': 4, '1991': 4, '2009': 4, '2896': 4, '2914': 4, '2930': 4, '2953': 4, '2964': 4, '2959': 4, '2979': 4, '2984': 4, '2027': 4, '3038': 4, '3048': 4

In [7]:
#Add number of algorithm kids  in the added column n_algokids in the copy of database ptcp_gnometrans 
for ind in ptcp_gnometrans.index:
    p_id=ptcp_gnometrans['participant_id'][ind]
    if str(int(p_id)) in n_algokids.keys():
        ptcp_gnometrans.at[ind,'n_algokids']=n_algokids[str(int(p_id))]

ptcp_gnometrans=ptcp_gnometrans.sort_values('n_algokids',ascending=False)
ptcp_gnometrans.to_csv('ptcp_gnometrans.csv',index=False)

# ptcp_gnometrans.head(10)

In [8]:
# #verify
# print("")
# print("TO VERIFY: number of algorithm kids of participant 2925 from the dictionary n_algokids:",n_algokids['2925'],", which is the same as shown in the dataframe")


In [15]:
#add column of fidelity to the database 
alpha=0.1
beta=1

for ind in ptcp_gnometrans.index:
    if ptcp_gnometrans['n_kids'][ind]==0: #when the participant has no kids, the probability will be set to zero
        ptcp_gnometrans.at[ind,'p_trans']=0
    else:
        ptcp_gnometrans.at[ind,'p_trans']=(ptcp_gnometrans['n_algokids'][ind]+alpha)/(ptcp_gnometrans['n_kids'][ind]+alpha+beta)

ptcp_gnometrans=ptcp_gnometrans.sort_values(by=['p_trans','n_algokids'],ascending=False) #sort by p_trans then by n_algokids
# ptcp_gnometrans.to_csv('ptcp_gnometrans.csv',index=False)

# ptcp_gnometrans.head(10)

In [16]:
def gnomeSort_qg(array):
    at=copy.copy(array)
    ii_f=0
    nn=0
    while nn<30:
        # print("ii=",ii_f)
        if at[ii_f]<at[ii_f+1]:
            # print("correct ofer and keep going forward")
            nn+=1
            # print("nn",nn)
            ii_f+=1
        else:
            # print("switch when going forward")
            # print(at)
            at[ii_f],at[ii_f+1]=at[ii_f+1],at[ii_f]
            # print(at)
            

            nn+=1
            # print("nn",nn)
            ii_b=ii_f
            ii_f+=1

            while nn<30:
                # print("ii =",ii_b)
                if ii_b==0:
                    # print("back to position zero")
                    break
                if at[ii_b-1]>at[ii_b]:
                    # print("switch and keep going back")
                    # print(at)
                    at[ii_b-1],at[ii_b]=at[ii_b],at[ii_b-1]
                    # print(at)
                    nn+=1
                    # print("nn",nn)
                    ii_b-=1
                if at[ii_b-1]<at[ii_b]:
                    # print("stop going back")
                    # print("ii_b",ii_b)
                    if ii_b==0:
                        break
                    else:
                        nn+=1
                        # print("nn",nn)
                        break
                # print(at)
        # print(at)
        if (at==np.array([1,2,3,4,5,6])).all():
            break
    # print(array)
    return array,nn




# #verify n_nece_compare
# arr_test=np.array([6,4,3,1,5,2]) #13
# arr_test=np.array([2,1,4,3,6,5]) #7
# arr_test=np.array([4,6,3,2,5,1]) #13

# print(gnomeSort_qg(arr_test))

In [17]:
def swap_ptcp(p_id,t_id):
    #input: participant id, trial id (trial 12 is demonstration)
    #output: number of necessary comparisons, score for the demonstration trial

    xx=trls[(trls['participant_id']==p_id)&(trls['trial_index']==t_id)]
    n_compare=xx['num_comparisons'].to_numpy()
    a=ord[(ord['participant_id']==p_id)&(ord['trial_index']==t_id)] #trial index 12 is the demonstration trial
    ord_ini=a['image_rank'].to_numpy()[0:6] #first six show the intial order
    ord_final=a['image_rank'].to_numpy()[6:12] #last six show the final order
    [order_ini,n_nece_compare]=gnomeSort_qg(ord_ini)


    #demonstration score
    if (ord_final == [1,2,3,4,5,6]).all():
        # if n_compare==n_nece_compare:
        #     s=1
        # else:
        #     s=math.exp(-n_compare/n_nece_compare+1) #not using this cus it leads to s larger than one when n_compare is smaller than n_nece_compare
        s=math.exp(-abs(n_compare-n_nece_compare))
    else:
        s=0

    return n_nece_compare,n_compare,s,order_ini


In [18]:
#verify
[n_nece_compare,n_compare,s,ord_ini]=swap_ptcp(3599,12)
print("initial order is:",ord_ini)
# print("final order is:",ord_final)
print("number of necessary comparisons:",n_nece_compare)
print("number of actul comparisons:",n_compare)
print("score of demonstration:",s)

initial order is: [6 1 5 4 3 2]
number of necessary comparisons: 15
number of actul comparisons: [11]
score of demonstration: 0.01831563888873418


In [19]:
#add column of "n_nece_compare" to show the number of necessary comparisons for each participant
for ind in ptcp_gnometrans.index:
    p_id=ptcp_gnometrans['participant_id'][ind]
    [n_nece_compare,n_compare,s,order_ini]=swap_ptcp(p_id,12)
    ptcp_gnometrans.at[ind,'n_nece_compare']=n_nece_compare
    ptcp_gnometrans.at[ind,'s_demo']=s

ptcp_gnometrans=ptcp_gnometrans.sort_values(by=['p_trans','n_algokids'],ascending=False) #sort by p_trans then by n_algokids
# ptcp_gnometrans=ptcp_gnometrans.sort_values(by=['s_demo','p_trans'],ascending=False) #sort by p_trans then by n_algokids
ptcp_gnometrans.to_csv('ptcp_gnometrans.csv',index=False)

for ii in range(0,10):
    print(["The description for participant ",ptcp_gnometrans.iloc[ii]["participant_id"],": ",ptcp_gnometrans.iloc[ii]["algorithm_description"]])


ptcp_gnometrans.head(10)

['The description for participant ', 2894, ': ', "I clicked on the second tile from the left, and then the first (furthest left). Then whether they swapped or not, I next clicked on the third tile first and then the second. If they switched, I then clicked on the new second tile, and then the first. Whenever there is either no switch made, or the switch is made so that the tile furthest left changes, I would move down one further right from where I started the last time and start with that, then click the one next to it. If the one next to it switches, I would click that new one (move one tile over left), and then click the one to the left of that one next to see if it switches. Whenever it doesn't switch, move back right and go one further to the right than what you started with last time. "]
['The description for participant ', 2605, ': ', "1. Work from left to right\n2. Compare the left most image with the one next to it\n3. Compare the 2nd image with the 3rd image\n3a. If they swap

,participant_id,n_kids,n_algokids,p_trans,n_nece_compare,s_demo,network_id,replication,generation,condition,cloned,mean_trial_score,algorithm,algorithm_description,exclusion_flag
2375,2894,6,6,0.859155,13,1.000000,8,8,7,SSL,False,0.778702,1.0,"I clicked on the second tile from the left, an...",False
2080,2605,6,6,0.859155,12,1.000000,5,5,6,SSL,False,0.781791,1.0,1. Work from left to right\n2. Compare the lef...,False
1187,2902,6,6,0.859155,11,1.000000,9,9,3,SSL,False,0.791802,1.0,Start on the 2nd picture always and then pick ...,False
2380,2969,6,6,0.859155,10,0.367879,9,9,7,SSL,False,0.799323,1.0,start with switching 2 and 1\nthen go to switc...,False
1796,2757,6,6,0.859155,13,1.000000,7,7,5,SSL,False,0.876795,1.0,Start with image 2 and then click on image 1. ...,False
1492,2349,5,5,0.836066,11,1.000000,2,2,4,SSL,False,0.853066,1.0,Work through positions #1-6 (left to right) un...,False
2098,2866,5,5,0.836066,12,0.367879,8,8,6,SSL,False,0.895491,1.0,Basic insertion sort. Select an index i=1 (the...,False
3252,3324,5,5,0.836066,10,1.000000,2,2,10,SSL,False,0.668273,1.0,I started with the 2nd picture from the left e...,False
2393,3074,5,5,0.836066,6,0.367879,10,10,7,SSL,False,0.870275,1.0,Tried to move pictures from left to right. Tri...,False
1795,2925,10,9,0.819820,9,0.367879,9,9,5,SSL,False,0.872616,1.0,1. Choose 2nd picture from the left and 1st pi...,False


In [20]:
#Generate a database for mixed effect model

# p_id_gnome=ptcp_gnometrans['participant_id'].to_numpy() #list of participant id that uses gnome algorithm
# data = [['tom', 10], ['nick', 15], ['juli', 14]]

i=0
pair_mix=[]
for ind in ntwk.index:
    l=[]
    p_id=ntwk['parent_participant_id'][ind]
    if p_id in p_id_gnome: #if parent uses gnome algorithm
        c_id=ntwk['participant_id'][ind]
        l.append(p_id) #parent id
        l.append(c_id) #children id
        if c_id in p_id_gnome: #if children uses gnome algorithm
            l.append(1) #T=1
        else:
            l.append(0) #T=0
        l.append(int(ptcp.loc[ptcp['participant_id'] ==p_id, 'generation'])) #generation
        l.append(float(ptcp_gnometrans.loc[ptcp_gnometrans['participant_id'] ==p_id, 's_demo'])) #s_demo
        pair_mix.append(l)


# Create the pandas DataFrame
pair_mix_df = pd.DataFrame(pair_mix, columns=['Parent ID', 'Child ID','Transmitted','Generation','S_demo'])
pair_mix_df.to_csv('transmission_pair_for_mixed_effect.csv',index=False)